<a href="https://colab.research.google.com/github/tiffanyjxw/montblanc/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1**

Use the NewsAPI to get URLs for news articles. Go to https://newsapi.org, create an account, and get a key.
Use the `https://newsapi.org/v2/everything` endpoint, which takes as input a keyword, queries the NewsAPI, and returns back a list of URLs with the news stories that contain the keyword.
Follow the documentation at https://newsapi.org/#documentation and figure out how to get back the news.





In [101]:
import requests

api_url = "https://newsapi.org/v2/everything"

keyword = input("Enter Keyword ")
parameters = {
    'q'       : keyword,
    'from'    : '2022-09-09',
    'language': 'en',
    'sortBy'  : 'publishedAt',
    'apiKey'  : '051aae6a370148a0aa1c755a12b911c6'
}
resp = requests.get(api_url, params=parameters)
data = resp.json()
data

Enter Keyword raytheon


{'status': 'ok',
 'totalResults': 26,
 'articles': [{'source': {'id': None, 'name': 'Truthout'},
   'author': 'Jamila Osman',
   'title': 'African People Bear the Weight of US’s Deadly “War on Terror” on the Continent',
   'description': "The post-9/11 creation of “AFRICOM” expanded the U.S.'s military footprint, spreading instability and violence.",
   'url': 'https://truthout.org/articles/african-people-bear-the-weight-of-uss-deadly-war-on-terror-on-the-continent/',
   'urlToImage': 'https://truthout.org/wp-content/uploads/2022/09/GettyImages-1241418925-1200x675.jpg',
   'publishedAt': '2022-09-11T16:02:55Z',
   'content': 'Skip to main content\r\nSkip to footer\r\nOn the evening of September 20, 2001, then-President George W. Bush addressed the American public and laid the political, military and ideological groundwork for… [+8012 chars]'},
  {'source': {'id': None, 'name': 'Space Daily'},
   'author': None,
   'title': 'Ocean lidar remote sensing technology based on Brillouin scatt

**Part 2**


Write code that stores the retrieved news articles in a database. You can use the db.ipeirotis.org MySQL server and create your table under the public database.

Please prefix with your netID all the tables you create in the public database. So, if you want to create a table called news and your netID is ab123, call the table ab123_news.

You will need to figure out which fields you want to save in the database, and their data types, create the appropriate table, and then insert in the database the news entries that you retrieved in Part 1. 


In [102]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [103]:
import uuid
from datetime import date, datetime, timedelta

In [104]:
news_articles = data['articles']
print(news_articles)

[{'source': {'id': None, 'name': 'Truthout'}, 'author': 'Jamila Osman', 'title': 'African People Bear the Weight of US’s Deadly “War on Terror” on the Continent', 'description': "The post-9/11 creation of “AFRICOM” expanded the U.S.'s military footprint, spreading instability and violence.", 'url': 'https://truthout.org/articles/african-people-bear-the-weight-of-uss-deadly-war-on-terror-on-the-continent/', 'urlToImage': 'https://truthout.org/wp-content/uploads/2022/09/GettyImages-1241418925-1200x675.jpg', 'publishedAt': '2022-09-11T16:02:55Z', 'content': 'Skip to main content\r\nSkip to footer\r\nOn the evening of September 20, 2001, then-President George W. Bush addressed the American public and laid the political, military and ideological groundwork for… [+8012 chars]'}, {'source': {'id': None, 'name': 'Space Daily'}, 'author': None, 'title': 'Ocean lidar remote sensing technology based on Brillouin scattering spectrum', 'description': 'Wuhan, China (SPX) Sep 12, 2022\n\n\nThe monito

In [105]:
from sqlalchemy import create_engine

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

In [106]:
news_articles[0]

{'source': {'id': None, 'name': 'Truthout'},
 'author': 'Jamila Osman',
 'title': 'African People Bear the Weight of US’s Deadly “War on Terror” on the Continent',
 'description': "The post-9/11 creation of “AFRICOM” expanded the U.S.'s military footprint, spreading instability and violence.",
 'url': 'https://truthout.org/articles/african-people-bear-the-weight-of-uss-deadly-war-on-terror-on-the-continent/',
 'urlToImage': 'https://truthout.org/wp-content/uploads/2022/09/GettyImages-1241418925-1200x675.jpg',
 'publishedAt': '2022-09-11T16:02:55Z',
 'content': 'Skip to main content\r\nSkip to footer\r\nOn the evening of September 20, 2001, then-President George W. Bush addressed the American public and laid the political, military and ideological groundwork for… [+8012 chars]'}

In [107]:
news_articles[1]['source']['name']

'Space Daily'

In [108]:
db_name = "public"

create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

engine.execute(create_db_query)

In [109]:
# Please prefix with your netID all the tables you create in the public database. 
# So, if you want to create a table called news and your netID is ab123, call the table ab123_news.


netid = 'tjw8831' 
table_name = f"{netid}_news"


In [110]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
engine.execute(drop_table_query)


In [111]:
#You will need to figure out which fields you want to save in the database, and their data types, create the appropriate table, and then insert in the database the news entries that you retrieved in Part 1.

# Create a table

create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (url varchar(250), 
                                PRIMARY KEY(url)
                                )"""
engine.execute(create_table_query)

In [112]:
query_template = f"""
                    INSERT IGNORE INTO  
                    {db_name}.{table_name}(url) 
                    VALUES (%s)
                  """

In [113]:
for entry in news_articles:
    url = entry["url"]
    query_parameters = (url)
    engine.execute(query_template, query_parameters)

In [114]:
results = engine.execute(f"SELECT * FROM {db_name}.{table_name}")
rows = results.fetchall()
results.close()

**Part 3**

Retrieve the news articles you stored in the database in Part 2, and use the IBM Watson Natural Language Understanding API to extract the sentiment of each of the news articles.



In [115]:
#API = LPkFHhc_fOFb4gS9t3Lb8DHuhTnGWNP9obbEHHHXPOpb

In [116]:
URL = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/307d1bc8-f1aa-42e9-91e6-c23c4edcd779'

API_KEY = 'LPkFHhc_fOFb4gS9t3Lb8DHuhTnGWNP9obbEHHHXPOpb'

def analyzeText(text=None, url=None):

    endpoint = f"{URL}/v1/analyze"
    username = "apikey"
    password = API_KEY
    
    parameters = {
        'features': 'sentiment',
        'version' : '2022-04-07',
        'text': text,
        'language' : 'en',
        'url' : url 
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    return resp.json()

In [117]:
for row in rows:
    print("Sentiment: ", analyzeText(url=row['url'])['sentiment'], "for url: ", row['url'])

Sentiment:  {'document': {'score': 0.741153, 'label': 'positive'}} for url:  http://www.aero-news.net/index.cfm?do=main.textpost&id=defee1d4-61e2-4894-82f5-6d47289fe1b1
Sentiment:  {'document': {'score': -0.3349, 'label': 'negative'}} for url:  https://finance.yahoo.com/news/active-protection-system-market-worth-113000347.html
Sentiment:  {'document': {'score': 0.689738, 'label': 'positive'}} for url:  https://finance.yahoo.com/news/insights-maritime-information-global-market-090300449.html
Sentiment:  {'document': {'score': 0.857495, 'label': 'positive'}} for url:  https://finance.yahoo.com/news/md-helicopters-announces-ownership-leadership-110500594.html
Sentiment:  {'document': {'score': 0.81595, 'label': 'positive'}} for url:  https://finance.yahoo.com/news/worldwide-electronic-flight-instrument-system-094300722.html
Sentiment:  {'document': {'score': -0.347849, 'label': 'negative'}} for url:  https://freerepublic.com/focus/f-news/4092320/posts
Sentiment:  {'document': {'score': -0

**Part 4**

Store in the database the sentiment of each news article in a new table. The table should contain just two fields, url and sentiment_score

In [118]:
new_table_name = f"{netid}_url_sentiment"

In [119]:

drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{new_table_name}"
engine.execute(drop_table_query)

In [120]:
create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{new_table_name} 
                                (url varchar(250),
                                sentiment float, 
                                PRIMARY KEY(url)
                                )"""
engine.execute(create_table_query)

In [121]:
query_template = f"""
                    INSERT IGNORE INTO 
                    {db_name}.{new_table_name}(url, sentiment) 
                    VALUES (%s, %s)
                  """

In [122]:
for row in rows:
    data = (analyzeText(url=row['url']))
    url = data['retrieved_url']
    sentiment = data['sentiment']['document']['score']
    print("Inserting sentiment", sentiment, "at", url)


    query_parameters = (url, sentiment)
    engine.execute(query_template, query_parameters)

Inserting sentiment 0.741153 at http://www.aero-news.net/index.cfm?do=main.textpost&id=defee1d4-61e2-4894-82f5-6d47289fe1b1
Inserting sentiment -0.3349 at https://finance.yahoo.com/news/active-protection-system-market-worth-113000347.html
Inserting sentiment 0.689738 at https://finance.yahoo.com/news/insights-maritime-information-global-market-090300449.html
Inserting sentiment 0.857495 at https://finance.yahoo.com/news/md-helicopters-announces-ownership-leadership-110500594.html
Inserting sentiment 0.81595 at https://finance.yahoo.com/news/worldwide-electronic-flight-instrument-system-094300722.html
Inserting sentiment -0.347849 at https://freerepublic.com/focus/f-news/4092320/posts
Inserting sentiment -0.458299 at https://freerepublic.com/focus/f-news/4092369/posts
Inserting sentiment -0.591572 at https://truthout.org/articles/african-people-bear-the-weight-of-uss-deadly-war-on-terror-on-the-continent/
Inserting sentiment -0.447502 at https://www.avclub.com/queen-elizabeth-ii-crazy-f